# Assistant-10<br/>Knowledge Retrieval

Notes:
- As of 01/14/2024 there's still a bug with knowledge retrieval.


### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import time
import oaihelper as framework

## Get an OpenAI client
client = framework.get_openai_client(api_key=framework.api_KEY,
        api_version=framework.api_version,
        azure_endpoint=framework.api_URI,)

In [2]:
## Files
file_benefits = framework.upload_file(client,"../data/contoso-benefits.txt")
file_401k = framework.upload_file(client,"../data/contoso-401k.txt")
file_leave = framework.upload_file(client,"../data/contoso-medical_leave.txt")

In [3]:
tools_list = [        
        {
            "type": "function",
            "function": {
                "name": "send_email",
                "description": "Sends an email to a recipient(s).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "to": {
                            "type": "string",
                            "description": "The email(s) the email should be sent to."
                        },
                        "content": {
                            "type": "string",
                            "description": "The content of the email."
                        }
                    },
                    "required": ["to", "content"]
                }
            }
        },
        {"type","retrieval"}
    ]

In [4]:
hr_assistant = framework.create_assistant(client,
        name="HR Assistant",
        instructions="You HR support chatbot. Use your knowledge base to best respond to the employee queries. Be friendly, professional, and helpful. If you don't know the answer, please say, 'contact HR for more information.'",
        tools=[{"type": "retrieval"}],
        model=framework.gpt_deployment_name,
        file_ids=[file_benefits.id, file_401k.id, file_leave.id],)

Added assistant:  asst_fDhA3vyjqLnXRqCk9vhZURqF 1


In [5]:
# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you please email to alemor@microsoft.com the procedure to request medical leave?"
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=hr_assistant.id,
    instructions="Today's date and time is: " + framework.get_localized_datetime() + ".\n\n"
)

In [6]:
while True:
    # Wait for 1 seconds
    time.sleep(5)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    #print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        ## Cleanup
        print(client.beta.threads.delete(thread.id))
        print(client.beta.assistants.delete(hr_assistant.id))
        print(client.files.delete(file_benefits.id))
        print(client.files.delete(file_401k.id))
        print(client.files.delete(file_leave.id))

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            
            if func_name == "send_email":
                print("Sending email...")
                email_to = arguments['to']
                email_content = arguments['content']
                framework.send_email({'to': email_to,'content':email_content})
                
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": "Email sent"
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)

Waiting for the Assistant to process...
Waiting for the Assistant to process...
Assistant: I'm sorry, but I am not able to send emails. However, I have found the procedure for requesting medical leave in the document provided. You can copy the information below and email it to the intended recipient.

---

Subject: Procedure to Request Medical Leave

Dear [Recipient's Name],

I would like to inform you of the procedure to request medical leave as specified in Contoso’s Medical Leave and Leave of Absence Policy. Please find the details below:

- Notify their supervisor and the human resources department as soon as possible about their need for leave and the expected duration.
- Provide appropriate documentation to support their request, such as a doctor's note, a certificate of enrollment, a court summons, or a military order.
- Complete and submit the required forms to the human resources department before the start of their leave.
- Keep in touch with their supervisor and the human re